In [2]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
monthly = pd.read_csv("monthly.csv")
monthly

,Year,MonthNo,MonthName,Energy
0,1924,10,Oct,0.0
1,1924,11,Nov,7.7
2,1924,12,Dec,24.3
3,1925,1,Jan,61.8
4,1925,2,Feb,109.5
...,...,...,...,...
1099,2016,5,May,139.0
1100,2016,6,Jun,128.9
1101,2016,7,Jul,81.4
1102,2016,8,Aug,48.2


In [5]:
monthly['Month']=monthly['MonthNo']
monthly=monthly.drop('MonthNo',1)
monthly['MonthId']=monthly['Year']+monthly['Month']/100
monthly=monthly.set_index(pd.Index(monthly['MonthId']))
monthly=monthly.drop(['MonthId'],axis=1)

monthly['Day']=1
monthly['DateFrom']=pd.to_datetime(monthly[['Year','Month','Day']])
monthly=monthly.drop('Day',1)


monthly['MonthDecimal']=monthly['Year']+monthly['Month']/12
monthly['WaterYear']=monthly.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
monthly['WaterMonth']=monthly.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)


monthly

,Year,MonthName,Energy,Month,DateFrom,MonthDecimal,WaterYear,WaterMonth
MonthId,,,,,,,,
1924.10,1924,Oct,0.0,10,1924-10-01,1924.833333,1924,1
1924.11,1924,Nov,7.7,11,1924-11-01,1924.916667,1924,2
1924.12,1924,Dec,24.3,12,1924-12-01,1925.000000,1924,3
1925.01,1925,Jan,61.8,1,1925-01-01,1925.083333,1924,4
1925.02,1925,Feb,109.5,2,1925-02-01,1925.166667,1924,5
...,...,...,...,...,...,...,...,...
2016.05,2016,May,139.0,5,2016-05-01,2016.416667,2015,8
2016.06,2016,Jun,128.9,6,2016-06-01,2016.500000,2015,9
2016.07,2016,Jul,81.4,7,2016-07-01,2016.583333,2015,10


In [6]:
yearly=monthly[['WaterYear']].groupby('WaterYear').count()
yearly['YearDecimal']=yearly.index+1.25

yearly['Energy']=monthly[['WaterYear','Energy']].groupby('WaterYear').sum()
yearly['Energy_min']=monthly[['WaterYear','Energy']].groupby('WaterYear').min()
yearly['Energy_max']=monthly[['WaterYear','Energy']].groupby('WaterYear').max()
yearly

,YearDecimal,Energy,Energy_min,Energy_max
WaterYear,,,,
1924,1925.25,841.2,0.0,137.4
1925,1926.25,801.0,14.7,127.8
1926,1927.25,910.7,20.6,137.0
1927,1928.25,863.3,18.6,126.2
1928,1929.25,771.7,25.8,134.9
...,...,...,...,...
2011,2012.25,946.6,26.8,122.8
2012,2013.25,984.9,31.1,125.9
2013,2014.25,919.8,29.3,122.6


In [7]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')

calmonthly['Energy_min']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').min()
calmonthly['Energy_mean']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').mean()
calmonthly['Energy_median']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').median()
calmonthly['Energy_max']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').max()


calmonthly

,MonthName,Month,Energy_min,Energy_mean,Energy_median,Energy_max
WaterMonth,,,,,,
1,Oct,10,0.0,28.659783,28.10,55.5
2,Nov,11,5.0,27.920652,26.65,61.1
3,Dec,12,20.3,46.028261,45.35,89.4
4,Jan,1,36.7,74.801087,70.10,126.6
5,Feb,2,46.3,90.465217,90.80,117.5
6,Mar,3,0.0,108.032609,115.20,137.4
7,Apr,4,0.0,99.172826,104.85,133.0
8,May,5,43.0,112.230435,113.35,139.7
9,Jun,6,50.1,115.981522,122.80,131.3


In [8]:
calmonthly_ps=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep']}, index=[10,11,12,1,2,3,4,5,6,7,8,9])

calmonthly_ps['P95']=monthly[['Month','Energy']].groupby('Month').quantile(0.05)
calmonthly_ps['P90']=monthly[['Month','Energy']].groupby('Month').quantile(0.1)
calmonthly_ps['P80']=monthly[['Month','Energy']].groupby('Month').quantile(0.2)
calmonthly_ps['P75']=monthly[['Month','Energy']].groupby('Month').quantile(0.25)
calmonthly_ps['P50']=monthly[['Month','Energy']].groupby('Month').quantile(0.5)
calmonthly_ps['P25']=monthly[['Month','Energy']].groupby('Month').quantile(0.75)
calmonthly_ps['P20']=monthly[['Month','Energy']].groupby('Month').quantile(0.8)
calmonthly_ps['P10']=monthly[['Month','Energy']].groupby('Month').quantile(0.9)
calmonthly_ps['P05']=monthly[['Month','Energy']].groupby('Month').quantile(0.95)

calmonthly_ps

,WaterMonth,MonthName,P95,P90,P80,P75,P50,P25,P20,P10,P05
10,1,Oct,15.565,17.32,21.14,22.875,28.10,34.625,35.62,42.13,45.270
11,2,Nov,13.235,17.21,20.70,21.450,26.65,32.975,35.78,40.66,46.235
12,3,Dec,27.340,30.91,35.02,36.375,45.35,53.450,54.10,63.53,69.665
1,4,Jan,51.365,54.11,58.84,61.625,70.10,87.150,91.76,102.28,110.930
2,5,Feb,64.620,69.33,77.10,80.175,90.80,103.300,106.88,112.80,115.890
3,6,Mar,51.675,82.03,97.56,99.350,115.20,123.600,125.46,127.38,129.680
4,7,Apr,43.600,67.54,81.86,85.275,104.85,119.125,120.00,126.12,130.090
5,8,May,82.395,91.36,100.82,102.375,113.35,125.200,127.80,133.06,135.675
6,9,Jun,72.460,96.71,110.82,114.925,122.80,126.800,127.18,127.90,128.790
7,10,Jul,45.940,54.21,66.62,71.575,90.00,108.800,113.98,122.43,127.495
